In [80]:
import pandas as pd
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [81]:
df = pd.read_csv("tiki__products.csv")

# Thêm cột 'platform' với giá trị 'tiki' cho toàn bộ các dòng
df["platform"] = "tiki"

In [82]:
df

,product_id,seller_product_id,url,product_name,sold,review_count,rating,price,currency,categories,sku,brand,seller_name,platform
0,278219887,278219889,https://tiki.vn/dien-thoai-oppo-reno13-f-5g-12...,Điện thoại OPPO Reno13 F 5G 12GB/512GB - Hàng ...,1,1,5.0,8490000.0,VND,Điện thoại Smartphone,5281905152791,OPPO,Tiki Trading,tiki
1,277737629,277737633,https://tiki.vn/dien-thoai-xiaomi-redmi-13-6gb...,Điện Thoại Xiaomi Redmi 13 6GB/128GB - Hàng Ch...,2,3,5.0,2899000.0,VND,Điện thoại Smartphone,9976795733949,Xiaomi,Hồng Hạnh Mobile,tiki
2,277651890,277651926,https://tiki.vn/dien-thoai-samsung-galaxy-a06-...,Điện thoại Samsung Galaxy A06 5G (4GB/128GB) -...,0,0,0.0,2789000.0,VND,Điện thoại Smartphone,6087287437900,Samsung,Long Hưng Official Store,tiki
3,277646748,277646755,https://tiki.vn/dien-thoai-samsung-galaxy-a06-...,Điện thoại Samsung Galaxy A06 5G (4GB/64GB) - ...,2,1,5.0,2539000.0,VND,Điện thoại Smartphone,7445595424087,Samsung,Long Hưng Official Store,tiki
4,277707642,277707647,https://tiki.vn/dien-thoai-samsung-galaxy-a26-...,Điện thoại Samsung Galaxy A26 5G (8GB/128GB) -...,0,0,0.0,5779000.0,VND,Điện thoại Smartphone,9058119526277,Samsung,Long Hưng Official Store,tiki
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,252137923,277339025,https://tiki.vn/bo-sac-truc-tiep-acc-cho-camer...,Bộ sạc trực tiếp Acc cho Camera hành trình Vie...,7,3,5.0,242000.0,VND,"Camera Hành Trình Ô Tô, Xe Máy",4641392159087,VietMap,VIETMAP,tiki
7325,128296737,128296764,https://tiki.vn/camera-hanh-trinh-sjcam-c200-h...,Camera hành trình SJCAM C200 - Hỗ trợ chống ru...,14,5,4.8,2195000.0,VND,"Camera Hành Trình Ô Tô, Xe Máy",6190388169259,SJCAM,Linh Kiện Store,tiki
7326,152844407,152844427,https://tiki.vn/camera-hanh-trinh-sjcam-sj5000...,Camera hành trình SJ5000X SJCAM - Actioncam qu...,6,3,4.7,2369000.0,VND,"Camera Hành Trình Ô Tô, Xe Máy",1538353331239,SJCAM,Linh Kiện Store,tiki
7327,9806421,55122169,https://tiki.vn/camera-hanh-trinh-sjcam-sj8-pr...,Camera Hành Trình SJCAM SJ8 Pro 4K Wifi - Hàng...,2,2,5.0,4065000.0,VND,"Camera Hành Trình Ô Tô, Xe Máy",1088264896712,SJCAM,The Gioi Dien May Shop,tiki


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import entropy

# Helper Functions

def preprocess_df(df):
    """Bỏ NA và thêm revenue_est"""
    df = df.dropna(subset=["price", "sold", "rating"])
    df["revenue_est"] = df["price"] * df["sold"]
    return df

def quantile_clip(series, q_low=0.01, q_high=0.99):
    """Cắt đuôi dài cho price/sold"""
    lower, upper = series.quantile(q_low), series.quantile(q_high)
    return series.clip(lower, upper)

def filter_A(df, A):
    """Lọc theo category hoặc keyword"""
    if A is None:
        return df
    if isinstance(A, str):
        mask = (
            df["categories"].str.contains(A, case=False, na=False)
            | df["product_name"].str.contains(A, case=False, na=False)
            | df["brand"].fillna("").str.contains(A, case=False)
        )
        return df[mask]
    elif isinstance(A, list):
        mask = df["categories"].isin(A) | df["brand"].isin(A)
        return df[mask]
    return df

def base_meta(filters, notes=""):
    return {
        "filters": filters,
        "notes": notes,
        "ts_generated": datetime.now().isoformat(timespec="seconds")
    }

# 1. describe_price()
def describe_price(df, A, platforms=None, by_platform=True, min_reviews=0):
    df = preprocess_df(df)
    df["price"] = quantile_clip(df["price"])
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    stats = []
    group_cols = ["platform"] if by_platform else [None]
    for key, g in df.groupby(group_cols) if by_platform else [(None, df)]:
        desc = {
            "platform": key if by_platform else "all",
            "count": len(g),
            "min_price": g["price"].min(),
            "q10": g["price"].quantile(0.1),
            "median_price": g["price"].median(),
            "mean_price": g["price"].mean(),
            "q90": g["price"].quantile(0.9),
            "max_price": g["price"].max(),
            "std_price": g["price"].std(),
        }
        stats.append(desc)

    return {
        "data": stats,
        "meta": base_meta(
            {"A": A, "platforms": platforms, "by_platform": by_platform},
            "Dropped NA (price/sold/rating), applied 1%-99% quantile clipping for price."
        )
    }

# 2. rating_distribution()
def rating_distribution(df, A, platforms=None, brands=None, bins=20, group_by_brand=True, min_reviews=0):
    df = preprocess_df(df)
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    if brands:
        df = df[df["brand"].isin(brands)]
    df = df[df["review_count"] >= min_reviews]

    bin_edges = np.linspace(df["rating"].min(), df["rating"].max(), bins + 1)
    data = []
    if group_by_brand:
        for brand, g in df.groupby("brand"):
            counts, _ = np.histogram(g["rating"], bins=bin_edges)
            for i in range(len(counts)):
                data.append({
                    "bucket_left": bin_edges[i],
                    "bucket_right": bin_edges[i+1],
                    "count": int(counts[i]),
                    "brand": brand
                })
    else:
        counts, _ = np.histogram(df["rating"], bins=bin_edges)
        for i in range(len(counts)):
            data.append({
                "bucket_left": bin_edges[i],
                "bucket_right": bin_edges[i+1],
                "count": int(counts[i])
            })

    return {
        "data": data,
        "meta": base_meta(
            {"A": A, "platforms": platforms, "brands": brands, "bins": bins},
            "Dropped NA, rating histogram computed."
        ) | {"bin_edges": bin_edges.tolist()}
    }

# 3. sold_distribution()
def sold_distribution(df, A, platforms=None, bins=[0,10,50,100,500,1000,5000,10000], min_reviews=0):
    df = preprocess_df(df)
    df["sold"] = quantile_clip(df["sold"])
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    data = []
    for plat, g in df.groupby("platform"):
        counts, edges = np.histogram(g["sold"], bins=bins)
        total = counts.sum()
        for i in range(len(counts)):
            data.append({
                "platform": plat,
                "bin_left": int(edges[i]),
                "bin_right": int(edges[i+1]),
                "count": int(counts[i]),
                "pct": float(counts[i] / total * 100) if total else 0
            })

    return {
        "data": data,
        "meta": base_meta(
            {"A": A, "platforms": platforms},
            "Dropped NA, applied 1%-99% quantile clipping for sold."
        )
    }

# 4. category_count_plot()
def category_count_plot(df, platforms=None, top_k=None, min_reviews=0):
    df = preprocess_df(df)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    cat_counts = df["categories"].value_counts().reset_index()
    cat_counts.columns = ["categories", "product_count"]
    if top_k:
        cat_counts = cat_counts.head(top_k)

    return {
        "data": cat_counts.to_dict(orient="records"),
        "meta": base_meta({"platforms": platforms}, "Dropped NA before counting.")
    }

# 5. brand_share_chart()
def brand_share_chart(df, A, platforms=None, metric="sku", normalize=True, min_reviews=0):
    df = preprocess_df(df)
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    if metric == "revenue_est":
        agg = df.groupby(["platform", "brand"], as_index=False)["revenue_est"].sum()
        agg.rename(columns={"revenue_est": "value"}, inplace=True)
    else:
        agg = df.groupby(["platform", "brand"], as_index=False)["sku"].count()
        agg.rename(columns={"sku": "value"}, inplace=True)

    if normalize:
        agg["share_pct"] = agg.groupby("platform")["value"].apply(lambda x: 100 * x / x.sum())

    return {
        "data": agg.to_dict(orient="records"),
        "meta": base_meta(
            {"A": A, "metric": metric, "normalize": normalize},
            "Dropped NA, revenue_est = price*sold."
        )
    }

# 6. top_sellers()
def top_sellers(df, A, platforms=None, by="product_count", top_k=20, min_reviews=0):
    df = preprocess_df(df)
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    if by == "sold":
        agg = df.groupby(["platform", "seller_name"], as_index=False)["sold"].sum()
        agg.rename(columns={"sold": "value"}, inplace=True)
    else:
        agg = df.groupby(["platform", "seller_name"], as_index=False)["sku"].count()
        agg.rename(columns={"sku": "value"}, inplace=True)

    agg = agg.sort_values("value", ascending=False).head(top_k)
    agg["rank"] = range(1, len(agg) + 1)

    return {
        "data": agg[["rank", "seller_name", "platform", "value"]].to_dict(orient="records"),
        "meta": base_meta({"A": A, "by": by}, "Dropped NA")
    }

# 7. top_brands()
def top_brands(df, A, platforms=None, by="revenue_est", top_k=20, min_reviews=0):
    df = preprocess_df(df)
    df = filter_A(df, A)
    if platforms:
        df = df[df["platform"].isin(platforms)]
    df = df[df["review_count"] >= min_reviews]

    if by == "sold":
        agg = df.groupby("brand", as_index=False)["sold"].sum()
    else:
        agg = df.groupby("brand", as_index=False)["revenue_est"].sum()

    agg.rename(columns={by: "value"}, inplace=True)
    agg = agg.sort_values("value", ascending=False).head(top_k)
    agg["rank"] = range(1, len(agg) + 1)

    return {
        "data": agg[["rank", "brand", "value"]].to_dict(orient="records"),
        "meta": base_meta({"A": A, "by": by}, "Dropped NA, revenue_est = price*sold.")
    }

# 8. seller_diversity_index()
def seller_diversity_index(df, platforms=None, min_products=5):
    df = preprocess_df(df)
    if platforms:
        df = df[df["platform"].isin(platforms)]

    data = []
    for (plat, seller), g in df.groupby(["platform", "seller_name"]):
        cat_counts = g["categories"].value_counts(normalize=True)
        div_idx = entropy(cat_counts)
        if len(g) >= min_products:
            data.append({
                "seller_name": seller,
                "platform": plat,
                "product_count": len(g),
                "unique_categories": g["categories"].nunique(),
                "diversity_index": div_idx
            })

    return {
        "data": data,
        "meta": base_meta({"platforms": platforms, "min_products": min_products},
                          "Shannon entropy computed over category distribution.")
    }

# 9. price_range_by_category()
def price_range_by_category(df, categories=None, platforms=None, brand=None, sku=None, quantiles=(0.1, 0.9), min_reviews=0):
    df = preprocess_df(df)
    df["price"] = quantile_clip(df["price"])
    if categories:
        df = df[df["categories"].isin(categories)]
    if platforms:
        df = df[df["platform"].isin(platforms)]
    if brand:
        df = df[df["brand"] == brand]
    if sku:
        df = df[df["sku"] == sku]
    df = df[df["review_count"] >= min_reviews]

    data = []
    for (plat, cat), g in df.groupby(["platform", "categories"]):
        data.append({
            "categories": cat,
            "platform": plat,
            "min_price": g["price"].min(),
            "q_low": g["price"].quantile(quantiles[0]),
            "median_price": g["price"].median(),
            "q_high": g["price"].quantile(quantiles[1]),
            "max_price": g["price"].max(),
            "count": len(g)
        })

    return {
        "data": data,
        "meta": base_meta(
            {"categories": categories, "platforms": platforms},
            "Dropped NA, applied 1%-99% quantile clipping for price."
        )
    }


In [84]:
res1 = describe_price(df, A="Điện thoại Smartphone")
print(json.dumps(res1, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "platform": "tiki",
      "count": 165,
      "min_price": 220000.0,
      "q10": 2357600.0,
      "median_price": 3589000.0,
      "mean_price": 6277605.636363637,
      "q90": 12909999.999999993,
      "max_price": 40790000.0,
      "std_price": 7345505.595228588
    }
  ],
  "meta": {
    "filters": {
      "A": "Điện thoại Smartphone",
      "platforms": null,
      "by_platform": true
    },
    "notes": "Dropped NA (price/sold/rating), applied 1%-99% quantile clipping for price.",
    "ts_generated": "2025-11-06T19:02:00"
  }
}


C:\Users\Thuy Quynh\AppData\Local\Temp\ipykernel_2352\141507573.py:53: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for key, g in df.groupby(group_cols) if by_platform else [(None, df)]:


In [85]:
res2 = rating_distribution(df, A="Điện thoại Smartphone",brands=["Samsung"], bins=10)
print(json.dumps(res2, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "bucket_left": 0.0,
      "bucket_right": 0.5,
      "count": 27,
      "brand": "Samsung"
    },
    {
      "bucket_left": 0.5,
      "bucket_right": 1.0,
      "count": 0,
      "brand": "Samsung"
    },
    {
      "bucket_left": 1.0,
      "bucket_right": 1.5,
      "count": 1,
      "brand": "Samsung"
    },
    {
      "bucket_left": 1.5,
      "bucket_right": 2.0,
      "count": 0,
      "brand": "Samsung"
    },
    {
      "bucket_left": 2.0,
      "bucket_right": 2.5,
      "count": 1,
      "brand": "Samsung"
    },
    {
      "bucket_left": 2.5,
      "bucket_right": 3.0,
      "count": 0,
      "brand": "Samsung"
    },
    {
      "bucket_left": 3.0,
      "bucket_right": 3.5,
      "count": 0,
      "brand": "Samsung"
    },
    {
      "bucket_left": 3.5,
      "bucket_right": 4.0,
      "count": 1,
      "brand": "Samsung"
    },
    {
      "bucket_left": 4.0,
      "bucket_right": 4.5,
      "count": 0,
      "brand": "Samsung"
    },
    

In [86]:
res3 = sold_distribution(df, A="Điện thoại Smartphone", bins=[0,10,50,100,500,1000])
print(json.dumps(res3, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "platform": "tiki",
      "bin_left": 0,
      "bin_right": 10,
      "count": 131,
      "pct": 79.39393939393939
    },
    {
      "platform": "tiki",
      "bin_left": 10,
      "bin_right": 50,
      "count": 19,
      "pct": 11.515151515151516
    },
    {
      "platform": "tiki",
      "bin_left": 50,
      "bin_right": 100,
      "count": 4,
      "pct": 2.4242424242424243
    },
    {
      "platform": "tiki",
      "bin_left": 100,
      "bin_right": 500,
      "count": 11,
      "pct": 6.666666666666667
    },
    {
      "platform": "tiki",
      "bin_left": 500,
      "bin_right": 1000,
      "count": 0,
      "pct": 0.0
    }
  ],
  "meta": {
    "filters": {
      "A": "Điện thoại Smartphone",
      "platforms": null
    },
    "notes": "Dropped NA, applied 1%-99% quantile clipping for sold.",
    "ts_generated": "2025-11-06T19:02:01"
  }
}


In [87]:
res4 = category_count_plot(df, top_k=5)
print(json.dumps(res4, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "categories": "Camera IP",
      "product_count": 1115
    },
    {
      "categories": "Máy Chấm Công",
      "product_count": 579
    },
    {
      "categories": "Nhãn In",
      "product_count": 533
    },
    {
      "categories": "Loa Bluetooth",
      "product_count": 373
    },
    {
      "categories": "Linh Kiện và Phụ Kiện Máy In",
      "product_count": 251
    }
  ],
  "meta": {
    "filters": {
      "platforms": null
    },
    "notes": "Dropped NA before counting.",
    "ts_generated": "2025-11-06T19:02:01"
  }
}


In [88]:
res5 = brand_share_chart(df, A="Điện thoại Smartphone", metric="sku")
print(json.dumps(res5, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "platform": "tiki",
      "brand": "Apple",
      "value": 5,
      "share_pct": 3.0303030303030303
    },
    {
      "platform": "tiki",
      "brand": "OPPO",
      "value": 28,
      "share_pct": 16.96969696969697
    },
    {
      "platform": "tiki",
      "brand": "OnePlus ",
      "value": 1,
      "share_pct": 0.6060606060606061
    },
    {
      "platform": "tiki",
      "brand": "POCO",
      "value": 1,
      "share_pct": 0.6060606060606061
    },
    {
      "platform": "tiki",
      "brand": "ROGTZ",
      "value": 1,
      "share_pct": 0.6060606060606061
    },
    {
      "platform": "tiki",
      "brand": "Realme",
      "value": 20,
      "share_pct": 12.121212121212121
    },
    {
      "platform": "tiki",
      "brand": "Samsung",
      "value": 44,
      "share_pct": 26.666666666666668
    },
    {
      "platform": "tiki",
      "brand": "Saramonic",
      "value": 1,
      "share_pct": 0.6060606060606061
    },
    {
      "platform": 

C:\Users\Thuy Quynh\AppData\Local\Temp\ipykernel_2352\141507573.py:177: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  agg["share_pct"] = agg.groupby("platform")["value"].apply(lambda x: 100 * x / x.sum())


In [89]:
res6 = top_sellers(df, A="Điện thoại Smartphone", by="sold", top_k=10)
print(json.dumps(res6, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "rank": 1,
      "seller_name": "Tiki Trading",
      "platform": "tiki",
      "value": 47746
    },
    {
      "rank": 2,
      "seller_name": "THẮNG THAO MOBILE",
      "platform": "tiki",
      "value": 472
    },
    {
      "rank": 3,
      "seller_name": "Hồng Hạnh Mobile",
      "platform": "tiki",
      "value": 162
    },
    {
      "rank": 4,
      "seller_name": "Long Hưng Official Store",
      "platform": "tiki",
      "value": 10
    },
    {
      "rank": 5,
      "seller_name": "Binaishop",
      "platform": "tiki",
      "value": 1
    },
    {
      "rank": 6,
      "seller_name": "Shop HG",
      "platform": "tiki",
      "value": 1
    },
    {
      "rank": 7,
      "seller_name": "vivo Official Store",
      "platform": "tiki",
      "value": 1
    },
    {
      "rank": 8,
      "seller_name": "CLICKBUY HCM",
      "platform": "tiki",
      "value": 0
    },
    {
      "rank": 9,
      "seller_name": "O-TECH SHOP",
      "platform": 

In [90]:
res7 = top_brands(df, A="Điện thoại Smartphone", by="revenue_est", top_k=10)
print(json.dumps(res7, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "rank": 1,
      "brand": "Apple",
      "value": 128183280000.0
    },
    {
      "rank": 2,
      "brand": "OPPO",
      "value": 122743365000.0
    },
    {
      "rank": 3,
      "brand": "Xiaomi",
      "value": 27136178000.0
    },
    {
      "rank": 4,
      "brand": "Samsung",
      "value": 3154824000.0
    },
    {
      "rank": 5,
      "brand": "Realme",
      "value": 878923000.0
    },
    {
      "rank": 6,
      "brand": "Tecno",
      "value": 232860000.0
    },
    {
      "rank": 7,
      "brand": "Vivo",
      "value": 16787000.0
    },
    {
      "rank": 8,
      "brand": "POCO",
      "value": 10250000.0
    },
    {
      "rank": 9,
      "brand": "Saramonic",
      "value": 562000.0
    },
    {
      "rank": 10,
      "brand": "ROGTZ",
      "value": 260930.0
    }
  ],
  "meta": {
    "filters": {
      "A": "Điện thoại Smartphone",
      "by": "revenue_est"
    },
    "notes": "Dropped NA, revenue_est = price*sold.",
    "ts_gener

In [91]:
res8 = seller_diversity_index(df, min_products=5)
print(json.dumps(res8, ensure_ascii=False, indent=2))


{
  "data": [
    {
      "seller_name": "AKIA SmartHome Store",
      "platform": "tiki",
      "product_count": 8,
      "unique_categories": 1,
      "diversity_index": 0.0
    },
    {
      "seller_name": "AKauto Saigon Center",
      "platform": "tiki",
      "product_count": 6,
      "unique_categories": 1,
      "diversity_index": 0.0
    },
    {
      "seller_name": "AMA Store",
      "platform": "tiki",
      "product_count": 45,
      "unique_categories": 2,
      "diversity_index": 0.10656595882801999
    },
    {
      "seller_name": "AN PHAT COMPUTER",
      "platform": "tiki",
      "product_count": 411,
      "unique_categories": 26,
      "diversity_index": 2.3063569566877504
    },
    {
      "seller_name": "ANKER STORE",
      "platform": "tiki",
      "product_count": 9,
      "unique_categories": 3,
      "diversity_index": 0.9950269901795212
    },
    {
      "seller_name": "AOA",
      "platform": "tiki",
      "product_count": 24,
      "unique_categories": 2

In [92]:
res9 = price_range_by_category(df, categories=["Điện thoại Smartphone"], quantiles=(0.1, 0.9))
print(json.dumps(res9, ensure_ascii=False, indent=2))

{
  "data": [
    {
      "categories": "Điện thoại Smartphone",
      "platform": "tiki",
      "min_price": 220000.0,
      "q_low": 2389000.0,
      "median_price": 3590000.0,
      "q_high": 13150000.000000013,
      "max_price": 40790000.0,
      "count": 163
    }
  ],
  "meta": {
    "filters": {
      "categories": [
        "Điện thoại Smartphone"
      ],
      "platforms": null
    },
    "notes": "Dropped NA, applied 1%-99% quantile clipping for price.",
    "ts_generated": "2025-11-06T19:02:01"
  }
}
